# Generate Particle ID
This notebook is used to select a particle group (particle ID) from the estimates generated by the estimate phase of Spyral. If you haven't already run the estimate phase and try to use this notebook, it will fail! You should also be sure to set some configuration values ahead of running this notebook. In particular, the particle_id_file field should be set to a file name that you want to use to save a new particle ID cut!

First lets load our modules:

In [ ]:
import sys
sys.path.append('..')
from spyral_utils.plot import CutHandler, serialize_cut, Histogrammer
from spyral_utils.nuclear import NuclearDataMap
from spyral.core.workspace import Workspace
from spyral.core.config import load_config
from spyral.core.constants import DEG2RAD
from spyral.core.particle_id import load_particle_id

import polars as pl
import matplotlib.pyplot as plt
from matplotlib.widgets import PolygonSelector
from pathlib import Path
from json import load, dump

%matplotlib widget

RAD2DEG = 1.0/DEG2RAD

With our modules loaded we can now begin loading the current Spyral configuration and workspace.

In [ ]:
# Change the config file name to match your setup
config = load_config(Path("D:\\e20009_pc\\e20009_config.json"))
ws = Workspace(config.workspace)
nuclear_map = NuclearDataMap()

Here we'll set some parameters we'll use for the particle ID. You'll want to change these for your specific particle ID. *Make sure to modify this for your experiment!*

In [ ]:
# Set these!
pid_name = "proton_cut" # name given to PID
pid_z = 1 # atomic number
pid_a = 1 # mass number
pid_path = ws.get_gate_file_path(config.solver.particle_id_filename) # Path to which we will write our PID


Now we'll create some utility objects from spyral-utils that will help us with the plotting

In [ ]:
grammer = Histogrammer()
handler = CutHandler()

Now we'll add some histograms to the Histogrammer

In [ ]:
grammer.add_hist2d("particle_id", (400, 400), ((-10.0, 2.0e4), (-0.1, 3.0))) # Plot of dEdx vs. Brho (particle ID)
grammer.add_hist1d("ion_chamber", 4096, (0.0, 4096.0)) # Plot of ion chamber (beam ID)
grammer.add_hist2d("kinematics", (720, 400), ((0.0, 180.0), (0.0, 3.0))) # Plot of polar angle vs. Brho (kinematics)

With our histograms created, we're now ready to load the data and fill the histograms!

In [ ]:
for run in range(config.run.run_min, config.run.run_max+1):
    run_path = ws.get_estimate_file_path_parquet(run)
    if not run_path.exists():
        continue
    df = pl.read_parquet(run_path)
    # The below filter is optional. Filter the data on the ion chamber gate. Comment/Uncomment the line below to turn on/off the filter
    df = df.filter((pl.col('ic_amplitude') > config.solver.ic_min_val) & (pl.col('ic_amplitude') < config.solver.ic_max_val))
    grammer.fill_hist2d('particle_id', df.select('dEdx').to_numpy(), df.select('brho').to_numpy())
    grammer.fill_hist2d('kinematics', df.select('polar').to_numpy() * RAD2DEG, df.select('brho').to_numpy())
    grammer.fill_hist1d('ion_chamber', df.unique(subset=['event']).select('ic_amplitude').to_numpy())

With our data loaded and histograms filled, we are ready to plot! We'll use the plotly library to make some interactive histograms. First, the ion chamber

In [ ]:
ic = grammer.get_hist1d("ion_chamber")
fig, ax = plt.subplots(1,1)
ax.stairs(ic.counts, edges=ic.bins)
ax.set_title("Ion Chamber")
ax.set_xlabel("Amplitude (arb.)")
ax.set_ylabel("Counts")
fig.set_figwidth(8.0)

You should see a plot of the ion chamber above!

Now lets plot the interesting bit, the particle ID. We also need to bind our cut handler to the figure we make, so the code below is a tad more involved.

In [ ]:
pid_hist = grammer.get_hist2d("particle_id")
fig, ax = plt.subplots(1,1)
_ = PolygonSelector(ax, handler.mpl_on_select)
mesh = ax.pcolormesh(pid_hist.x_bins, pid_hist.y_bins, pid_hist.counts)
fig.colorbar(mesh, ax=ax)
ax.set_title("Particle ID")
ax.set_xlabel("dE/dx(arb.)")
ax.set_ylabel("B$\\rho$(Tm)")
fig.set_figheight(8.0)
fig.set_figwidth(11.0)


You should see the particle ID spectrum above! You can draw your cut by clicking the points to make the cut on the plot. Make sure to close the polygon by clicking the first point you made!

Lets also plot our kinematics

In [ ]:
kine = grammer.get_hist2d("kinematics")
fig, ax = plt.subplots(1,1)
mesh = ax.pcolormesh(kine.x_bins, kine.y_bins, kine.counts)
fig.colorbar(mesh, ax=ax)
ax.set_title("Kinematics")
ax.set_xlabel("Polar angle $\\theta$ (deg)")
ax.set_ylabel("B$\\rho$(Tm)")
fig.set_figheight(8.0)
fig.set_figwidth(11.0)

You should see a spectrum of the kinematics above!

Once you've done all of that and have *made* a selection in the particle ID, we want to save the particle ID to our workspace. Only run the cell below once you've made a selection!

In [ ]:
print(handler.cuts)

In [ ]:
# If you've made multiple cuts, you'll want to change the name used here. Cuts are automatically named in the order they were made (first cut is cut_0, second cut_1, etc.)
cut = handler.cuts["cut_0"]
cut.name = pid_name
serialize_cut(cut, pid_path)
cut_json = None
# Add the Z, A fields for particle ID
with open(pid_path, "r") as pid_file:
    cut_json =  load(pid_file)
    cut_json["Z"] = pid_z
    cut_json["A"] = pid_a
with open(pid_path, "w") as pid_file:
    dump(cut_json, pid_file)


Great! Now if you look in the PID file, you should see some JSON describing the cut!

As a test, we can now load the particle ID and apply it to itself and our kinematics. This helps us make sure that everything is working correctly. Again, only run this cell if you've already saved a particle ID!

In [ ]:
pid_cut = load_particle_id(pid_path, nuclear_map)
if pid_cut is None:
    raise ValueError("Load particle ID failed!")

grammer.add_hist2d("particle_id_gated", (400, 400), ((-100.0, 2.0e4), (0.0, 3.0))) # Plot of dEdx vs. Brho (particle ID), gated on PID
grammer.add_hist2d("kinematics_gated", (720, 400), ((0.0, 180.0), (0.0, 3.0))) # Plot of polar angle vs. Brho (kinematics), gated on PID

for run in range(config.run.run_min, config.run.run_max+1):
    run_path = ws.get_estimate_file_path_parquet(run)
    if not run_path.exists():
        continue
    df = pl.read_parquet(run_path)
    # The below filter is optional. Filter the data on the ion chamber gate. Comment/Uncomment the line below to turn on/off the filter
    df = df.filter((pl.col('ic_amplitude') > config.solver.ic_min_val) & (pl.col('ic_amplitude') < config.solver.ic_max_val))
    df = df.filter(pl.struct(['dEdx', 'brho']).map_batches(pid_cut.cut.is_cols_inside))
    print(f"run: {run} df: \n{df.select(['event', 'cluster_index', 'cluster_label', 'brho', 'eloss', 'cutoff_index'])}")
    grammer.fill_hist2d('particle_id_gated', df.select('dEdx').to_numpy(), df.select('brho').to_numpy())
    grammer.fill_hist2d('kinematics_gated', df.select('polar').to_numpy() * RAD2DEG, df.select('brho').to_numpy())

In [ ]:
pid_gated = grammer.get_hist2d("particle_id_gated")
fig, ax = plt.subplots(1,1)
mesh = ax.pcolormesh(pid_gated.x_bins, pid_gated.y_bins, pid_gated.counts)
fig.colorbar(mesh, ax=ax)
ax.set_title("Particle ID Gated")
ax.set_xlabel("dE/dx(arb.)")
ax.set_ylabel("B$\\rho$(Tm)")
fig.set_figheight(8.0)
fig.set_figwidth(11.0)

In [ ]:
kine_gated = grammer.get_hist2d("kinematics_gated")
fig, ax = plt.subplots(1,1)
mesh = ax.pcolormesh(kine_gated.x_bins, kine_gated.y_bins, kine_gated.counts)
fig.colorbar(mesh, ax=ax)
ax.set_title("Kinematics Gated")
ax.set_xlabel("Polar angle $\\theta$ (deg)")
ax.set_ylabel("B$\\rho$(Tm)")
fig.set_figheight(8.0)
fig.set_figwidth(11.0)

You should see the plots above with your cut applied! You now have a particle ID ready for use in the final phase of Spyral!